# This program find the decay rate after n days for given activity and also calculates the time required to get number of events for the new rate

# DECAY Rate Equation
# $ R(t)=R_{0}e^{-\lambda t} $
# $ t_{1/2}=\frac{ln(2)}{\lambda} $
# $ R(t)=R_{0}e^{-\frac{ln(2)\cdot t}{t_{1/2}}} $
- $ R_{0} $ (initial rate)
- $ R(t)$ (rate after t seconds)
- $ T_{1/2} $ (half life)



# Realistic rate approach
# $$ R(A) = \gamma A (1+\alpha A t)e^{-\alpha A t }$$


# STEPS
- get optimal rate $R_{0}$ from realistic rate equation (refer: th228rate.ipynb)
- get the decay rate after particular time (t) with decay rate equation considering the half life 
- get R(A) for subsequent rate decay rate R(t) where A is the decay rate using the same $\gamma$, $\alpha$, & $t$ used for  optimal equation values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Results from optimal rate

In [ ]:
alpha  = 0.06819796206
beta   = 0.0044755
gamma  = 1.768248e-05
t = 0.01 #drift time (10 ms)

In [ ]:
th228_half_life= 1.9116*365            #1.9116 year into days
th228_initial_activity=2137.32         #optimal activity from simulation of 5M events


print(f"""
Th228 half life       : {th228_half_life} d
Th228 optimal activity: {th228_initial_activity} Bq
""")



In [ ]:
def get_rate_after(initial_activity,half_life,t):
    '''
    returns the rate after time t  for given initial activity and half life: half_life
    Note: t and half_need to have the same unit
    
    '''
    exp_part=np.exp(- np.log(2)*t/half_life)
    return initial_activity*exp_part

#function to find the realistic rate
def get_realistic_rate(A,gamma,alpha,t):
    '''
    returns the realistic rate  for given A, alpha, gamma, and t
    '''
    com=alpha*A*t
    exp_part=np.exp(-com)
    # print(f"""
    # common  : {com}
    # exp_part: {exp_part}
    # """)
    #rate=gamma*A*(1+alpha*A*t)*math.exp(-alpha*A*t)
    rate=gamma*A*(1+com)*exp_part
    #print(f"rate : {rate}")
    return rate
    

In [ ]:
def get_all_plot(df_1,da,yr,save_fig=False):
    """
    returns three plots 
    """
    
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(18, 6))
    days=df_1['days(d)']
    
    ax0.plot(days,df_1['decay_rate(Bq)'])
    ax0.set_xlabel("days")
    ax0.set_ylabel('decay_rate [Bq]')
    
    ax1.plot(days,df_1['optimal_rate(Hz)'])
    ax1.set_xlabel('days')
    ax1.set_ylabel('Optimal_rate [Hz]')
    ax2.plot(days,df_1['100_events_time(hr)'])
    ax2.set_xlabel('days')
    ax2.set_ylabel('100_events_time [Hr]')
    
    plt.suptitle(f"""
    increment days: {da}
    total time    :{yr} yrs
    """)
    plt.plot()
    save_name=f'yr-{yr}-per-{da}.pdf'
    if save_fig:
        print(f"saving the figure with name: {save_name}")
        plt.savefig(save_name)
    plt.show()


In [ ]:
#single function

def get_all_single(increment_days=1,yrs=5,debug=False):
    """
    returns a dataframe with columns 'days(d)', 'decay_rate(Bq)', 'optimal_rate(Hz)', 'optimal_time(s)' and,
       '100_events_time(hr)' for increment_days and years.
    """
    days=[];rate=[];time=[]

    # 5yrs increment by 1 day
    print(f"""
    ===================================================
    Th228 half life       : {th228_half_life:10} d
    Th228 optimal activity: {th228_initial_activity:10} Bq
    ===================================================
    """)

    print(f"""
    Total time: {yrs} yrs
    increment : {increment_days} d

    """)
    for i in range(1,365*yrs,increment_days):
        rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
        days.append(i);rate.append(rate_realistic);
        debug=False
        if (i-1)%200==0 and debug==True:
            print(f"""
            day       : {i} day(s)
            decay rate: {rate_realistic} Hz
            time      : {t_realistic} hr
            """)
    #optimal rate
    optimal_rate=np.array([get_realistic_rate(i,gamma,alpha,t) for i in rate])
    
    #dataframe
    df_1=pd.DataFrame()
    df_1['days(d)']=days
    df_1['decay_rate(Bq)']=rate
    df_1['optimal_rate(Hz)']=optimal_rate
    df_1['optimal_time(s)']=1/df_1['optimal_rate(Hz)']
    df_1['100_events_time(hr)']=100*df_1['optimal_time(s)']/(60*60)
    #debug True or False
    if(debug):
        print(f"                                    DATA FRAME:\n{df_1.head().to_string()}")
        print(f".........................................")
        print(f"{df_1.tail().to_string()}")

    
    get_all_plot(df_1,increment_days,yrs,save_fig=True)
    
    #return df_1

In [ ]:
#[1]+list(range(15,5*365,15))

In [ ]:
#single function to find total deployment time

def get_total_deployment_time(increment_days=1,th228_initial_activity=2137.32,yrs=5,debug=False):
    """
    returns a dataframe with columns 'optimal_activity(Bq)','increment_days(d)', 'total_deployment_time(yr)','total_100_events_time(hr)'
       
    """
    days=[];rate=[];time=[]

    # 5yrs increment by increment_days 
    if(debug):print(f"""
    ===================================================
    Th228 half life       : {th228_half_life:10} d
    Th228 optimal activity: {th228_initial_activity:10} Bq
    ===================================================
    """)
    
    if debug:print(f"""
    Total time: {yrs} yrs
    increment : {increment_days} d

    """)

   
    for i in range(1,365*yrs,increment_days):
        rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
        days.append(i);rate.append(rate_realistic);
        # debug=False
        # if (i)%200==0 and debug==True:
        #     print(f"""
        #     day       : {i} day(s)
        #     decay rate: {rate_realistic} Hz
        #     time      : {t_realistic} hr
        #     """)
    #optimal rate
    optimal_rate=np.array([get_realistic_rate(i,gamma,alpha,t) for i in rate])
    
    #dataframe
    df_1=pd.DataFrame()
    df_1['days(d)']=days
    df_1['decay_rate(Bq)']=rate
    df_1['optimal_rate(Hz)']=optimal_rate
    df_1['optimal_time(s)']=1/df_1['optimal_rate(Hz)']
    df_1['100_events_time(hr)']=100*df_1['optimal_time(s)']/(60*60)
    
    if(debug):
        print(f"                                    DATA FRAME:\n{df_1.head().to_string()}")
        print(f".........................................")
        print(f"{df_1.tail().to_string()}")

    total_100_events_time=df_1['100_events_time(hr)'].sum().round(2)
    if(debug):print(f"""
        total_100_events_time:{total_100_events_time} hr

        """)
   

    
    #get_all_plot(df_1,increment_days,yrs,save_fig=True)
    return_list=[th228_initial_activity,increment_days,yrs,total_100_events_time]
    print(f"Return List: {return_list}")
    return return_list
    
    #return df_1

In [ ]:
# yrs=5 #total deployment yrs
# #days_list=[1]+list(range(15,365*yrs,15))
# days_list=[1,2,3,4,7,15,30]
# activity_list=list(range(500,10500,500))
# final_list=[]
# for act in activity_list:
#     for i in days_list:
#         final_list.append(get_total_deployment_time(increment_days=i,th228_initial_activity=act,yrs=yrs))

In [ ]:
# MULTIPLE RESET OF SOURCE

In [ ]:
def multiple_reset_df(reset_yrs,days_list):
    '''
    returns the single dataframe for given reset_yrs and increment days list
    '''
    activity_list=list(range(500,5500,500))
    final_list=[]
    for act in activity_list:
        for i in days_list:
            final_list.append(get_total_deployment_time(increment_days=i,th228_initial_activity=act,yrs=reset_yrs))
    df_f=pd.DataFrame(final_list,columns=['initial_activity(Bq)','increment_days(d)','reset_time(yr)','total_100_events_time(hr)'])
    # df_d=df_f.groupby('increment_days(d)')
    # for da in df_days:
    #     temp_df=df_d.get_group(da)#['total_100_events_time(hr)']
    #     min_val=temp_df['total_100_events_time(hr)'].min()
    #     min_idx=temp_df['total_100_events_time(hr)'].argmin()
    #     min_activity=temp_df['initial_activity(Bq)'].values[min_idx]
    #     #print(f"min_activity: {min_activity}")
    #     #min_
    #     #print(f"min_idx:{min_idx}")
    #     tit=f"""
    #         increment          : {da:} day(s)
    #         min_100_events_time: {min_val:} hr
    #         min_activity       : {min_activity:} Bq'
    #     """
    #     print(f"===\nINFO: {tit}\n===")
    #     # temp_df.plot(x='initial_activity(Bq)',y='total_100_events_time(hr)',\
    #     #              title=tit,
    #     #              style='r.',ylabel='100_events_time(hr)',figsize=(20,10))
    #     # temp_df.plot(x='initial_activity(Bq)',y='total_100_events_time(hr)',title=f'\
    #     # increment:{da:>20} day(s) \n          min_100_events_time: {min_val:>5} hr\nmin_activity: {min_activity:>10} Bq',\
    #     #              style='b.',ylabel='100_events_time(hr)',figsize=(20,10))
    #     plt.show()
    #     print(f"temp_df:\n{temp_df.to_string()}")
    #     min_val=df_d.get_group(df_days[0])['total_100_events_time(hr)'].min()
    return df_f
    

In [ ]:
#multiple_reset_df(1,days_list)

In [ ]:
#dataframes with days_list and reset_yr
days_list=[1,2,3,4,7,15,30]

df_1=multiple_reset_df(1,days_list)  #reset year and days list
df_2=multiple_reset_df(2,days_list)
df_3=multiple_reset_df(3,days_list)
df_4=multiple_reset_df(4,days_list)
df_5=multiple_reset_df(5,days_list)

# RESET INFO FUNCTION

In [ ]:
#for this consider the minimum of 1 year and multiply by 10

def get_reset_info(df,reset_yr):
    parameter='increment_days(d)'
    #reset_yr=1
    df_grouped=df.groupby(parameter)
    main_list=[]
    column_names=['reset_year(yr)','calibration_frequency(d)','min_activity(Bq)','min_100_events_time(hr)']
    for day in days_list:
        temp_df=df_grouped.get_group(day).reset_index(drop=True)
        print(f'{temp_df.to_string()}') #print the df

        min_val=temp_df['total_100_events_time(hr)'].min()
        min_idx=temp_df['total_100_events_time(hr)'].argmin()
        min_activity=temp_df['initial_activity(Bq)'].values[min_idx]
        
        d='day' if day<2 else 'days'

        tit=f"""
                reset_year           : {reset_yr}
                calibration frequency: {day:} {d}
                min_100_events_time  : {min_val:} hr
                min_activity         : {min_activity:} Bq
            """
        print(f"===\nINFO: {tit}\n===")
        #reset_yr=1
        tit=f"Total_100_events_time at various initial activity for reset_year {reset_yr} and calibration frequecy {day} {d}"
        temp_df.plot(x='initial_activity(Bq)',y='total_100_events_time(hr)',\
                 title=tit,
                 style='r.',ylabel='total_100_events_time(hr)',figsize=(20,10))
        save_name=f'total_{day}_{reset_yr}.pdf'
        print(f'save_name: {save_name}')
        plt.show()
        main_list.append([reset_yr,day,min_activity,min_val])
    sum_df=pd.DataFrame(main_list,columns=column_names)
    print(f'{sum_df.to_string()}')
    return sum_df

    #print(f'{temp_df.to_string()}')


In [ ]:
#all reset_infos
df_s=[df_1,df_2,df_3,df_4,df_5]
reset_yrs=[1,2,3,4,5]

reset_yr=1

df_summary=[get_reset_info(df_s[i],reset_yrs[i]) for i in range(5)]


# RESET SUMMARY

# 1 YEAR RESET

In [ ]:
sum_head=28*'=='+' {} YEAR SUMMARY '+28*'=='
total_yrs=10
ten_yr_hours=total_yrs*365*24
print(f"10_yrs_into_hours: {ten_yr_hours}")

In [ ]:
summary_1=df_summary[1-1].copy()
print(sum_head.format('1')+"\n")
print(f"{summary_1.to_string()}\n")
summary_1_10=summary_1.copy(deep=True)
summary_1_10['10yr_min_100_events_time(hr)']=summary_1_10['min_100_events_time(hr)']*10
summary_1_10['10yr_fraction(%)']=round(summary_1_10['10yr_min_100_events_time(hr)']*100/ten_yr_hours,2)
print(sum_head.format('10')+"\n")
print(f"{summary_1_10.to_string()}\n")
tit='10 year min 100 events time for various calibration days'
summary_1_10.plot(x='calibration_frequency(d)',y='10yr_min_100_events_time(hr)',\
                 title=tit,
                 style='r.',ylabel='10yr_min_100_events_time(hr)',figsize=(10,8))
plt.show()

# 2 YEAR RESET

In [ ]:
summary_2=df_summary[2-1].copy()
print(sum_head.format('2')+"\n")
print(f"{summary_2.to_string()}\n")
summary_2_10=summary_2.copy()
summary_2_10['10yr_min_100_events_time(hr)']=summary_2_10['min_100_events_time(hr)']*5
summary_2_10['10yr_fraction(%)']=round(summary_2_10['10yr_min_100_events_time(hr)']*100/ten_yr_hours,2)
print(sum_head.format('10')+"\n")
print(f"{summary_2_10.to_string()}\n")

# 3 YEAR RESET

In [ ]:
summary_3=df_summary[3-1].copy()
print(sum_head.format('3')+"\n")
print(f"{summary_3.to_string()}\n")
print(sum_head.format('1')+"\n")
print(f"{summary_1.to_string()}\n")
summary_3_10=summary_3.copy()
summary_3_10['10yr_min_100_events_time(hr)']=3*summary_3['min_100_events_time(hr)']+summary_1['min_100_events_time(hr)']
summary_3_10['10yr_fraction(%)']=round(summary_3_10['10yr_min_100_events_time(hr)']*100/ten_yr_hours,2)
# summary_2_10['10yr_min_100_events_time(hr)']=summary_2_10['min_100_events_time(hr)']*5
print(sum_head.format('10')+"\n")
print(f"{summary_3_10.to_string()}\n")


# 4 YEAR RESET

In [ ]:
summary_4=df_summary[4-1].copy()
print(sum_head.format('4')+"\n")
print(f"{summary_4.to_string()}\n")
print(sum_head.format('2')+"\n")
print(f"{summary_2.to_string()}\n")
summary_4_10=summary_4.copy()
summary_4_10['10yr_min_100_events_time(hr)']=2*summary_4['min_100_events_time(hr)']+summary_2['min_100_events_time(hr)']
summary_4_10['10yr_fraction(%)']=round(summary_4_10['10yr_min_100_events_time(hr)']*100/ten_yr_hours,2)
# # summary_2_10['10yr_min_100_events_time(hr)']=summary_2_10['min_100_events_time(hr)']*5
print(sum_head.format('10')+"\n")
print(f"{summary_4_10.to_string()}\n")

# 5 YEAR RESET

In [ ]:
summary_5=df_summary[5-1].copy()
print(sum_head.format('5')+"\n")
print(f"{summary_5.to_string()}\n")

summary_5_10=summary_5.copy()
summary_5_10['10yr_min_100_events_time(hr)']=2*summary_5_10['min_100_events_time(hr)']
summary_5_10['10yr_fraction(%)']=round(summary_5_10['10yr_min_100_events_time(hr)']*100/ten_yr_hours,2)
# # summary_2_10['10yr_min_100_events_time(hr)']=summary_2_10['min_100_events_time(hr)']*5
print(sum_head.format('10')+"\n")
print(f"{summary_5_10.to_string()}\n")

In [ ]:
#creating the plot with different reset year and calibration frequency
for i in [1,2,3,4,5,6,7]:
    day=i
    temp_df=pd.concat(
    [summary_1_10.head(day).tail(1),\
    summary_2_10.head(day).tail(1),\
    summary_3_10.head(day).tail(1),\
    summary_4_10.head(day).tail(1),\
    summary_5_10.head(day).tail(1)]).reset_index(drop=True)
    temp_df.drop(columns=['min_100_events_time(hr)'],inplace=True)
    temp_st=37*'==='
    
    fig, ax = plt.subplots(figsize=(20,10))
    

    print(f"{temp_st}\n{temp_df.to_string()}\n{temp_st}\n")
    fname=str(temp_df.head(1)['calibration_frequency(d)'].values[0])
    #print(f"fname: {fname}")
    test='calibration frequency: '+fname
    test=test+" day" if i==1 else test+" days"
    ylabel='10yr_min_100_events_time(hr)'
    sylabel='10yr_fraction(%)'
    temp_df.plot('reset_year(yr)','10yr_min_100_events_time(hr)',
                 style='rx',ylabel=ylabel,title=f"{test}",
                 figsize=(16,12),ax=ax)
    temp_df.plot('reset_year(yr)','10yr_fraction(%)',style='b.',ax=ax,secondary_y=True)
    ax.right_ax.set_ylabel(sylabel)
    ax.grid(linestyle=':',linewidth=1.5,alpha=0.5,which='both')
    save_name=f'frequency-{fname}.pdf'
    print(f"saving the result to file: {save_name}")
    plt.savefig(save_name)

    plt.show()

In [ ]:
#reset year
for i in [1,2,3,4,5,6,7]:
    day=i
    temp_df=pd.concat(
    [summary_1_10.head(day).tail(1),\
    summary_2_10.head(day).tail(1),\
    summary_3_10.head(day).tail(1),\
    summary_4_10.head(day).tail(1),\
    summary_5_10.head(day).tail(1)])
    temp_df.drop(columns=['min_100_events_time(hr)'],inplace=True)
    temp_st=35*'==='
    
    fig, ax = plt.subplots(figsize=(20,10))
    

    print(f"{temp_st}\n{temp_df.to_string()}\n{temp_st}\n")
    fname=str(temp_df.head(1)['calibration_frequency(d)'].values[0])
    #print(f"fname: {fname}")
    test='calibration frequency: '+fname
    test=test+" day" if i==1 else test+" days"
    ylabel='10yr_min_100_events_time(hr)'
    sylabel='10yr_fraction(%)'
    temp_df.plot('reset_year(yr)','10yr_min_100_events_time(hr)',
                 style='rx',ylabel=ylabel,title=f"{test}",
                 figsize=(16,12),ax=ax)
    temp_df.plot('reset_year(yr)','10yr_fraction(%)',style='b.',ax=ax,secondary_y=True)
    ax.right_ax.set_ylabel(sylabel)
    ax.grid(linestyle=':',linewidth=1.5,alpha=0.5,which='both')
    save_name=f'frequency-{fname}.pdf'
    print(f"saving the result to file: {save_name}")
    plt.savefig(save_name)

    plt.show()

In [ ]:
#one day, hours in 10 yrs, fraction in terms of 10yrs

In [ ]:
#for this consider the minimum of 1 year and multiply by 10
# parameter='increment_days(d)'
# reset_yr=1
# df_1_grouped=df_1.groupby(parameter)
# main_list=[]
# column_names=['reset_year(yr)','increment_day','min_100_events_time(hr)','min_activity(Bq)']
# for day in days_list:
#     temp_df=df_1_grouped.get_group(day)
#     print(f'{temp_df.to_string()}') #print the df
    
#     min_val=temp_df['total_100_events_time(hr)'].min()
#     min_idx=temp_df['total_100_events_time(hr)'].argmin()
#     min_activity=temp_df['initial_activity(Bq)'].values[min_idx]
    
#     tit=f"""
#             increment          : {day:} day(s)
#             min_100_events_time: {min_val:} hr
#             min_idx            : {min_idx}
#             min_activity       : {min_activity:} Bq
#         """
#     print(f"===\nINFO: {tit}\n===")
#     main_list.append([reset_yr,day,min_val,min_activity])
# sum_df=pd.DataFrame(main_list,columns=column_names)
# sum_df

    #print(f'{temp_df.to_string()}')


In [ ]:
# df_f=pd.DataFrame(final_list,columns=['initial_activity(Bq)','increment_days(d)','total_deployment_time(yr)','total_100_events_time(hr)'])
# df_f

In [ ]:
# group by increment_days
# df_days=list(df_f['increment_days(d)'].unique())
# df_days

In [ ]:
# df_d=df_f.groupby('increment_days(d)')
# for da in df_days:
#     temp_df=df_d.get_group(da)#['total_100_events_time(hr)']
#     min_val=temp_df['total_100_events_time(hr)'].min()
#     min_idx=temp_df['total_100_events_time(hr)'].argmin()
#     min_activity=temp_df['initial_activity(Bq)'].values[min_idx]
#     #print(f"min_activity: {min_activity}")
#     #min_
#     #print(f"min_idx:{min_idx}")
#     tit=f"""
# increment          : {da:} day(s)
# min_100_events_time: {min_val:} hr
# min_activity       : {min_activity:} Bq'
#     """
#     print(f"===\nINFO: {tit}\n===")
#     temp_df.plot(x='initial_activity(Bq)',y='total_100_events_time(hr)',\
#                  title=tit,
#                  style='r.',ylabel='100_events_time(hr)',figsize=(20,10))
#     # temp_df.plot(x='initial_activity(Bq)',y='total_100_events_time(hr)',title=f'\
#     # increment:{da:>20} day(s) \n          min_100_events_time: {min_val:>5} hr\nmin_activity: {min_activity:>10} Bq',\
#     #              style='b.',ylabel='100_events_time(hr)',figsize=(20,10))
#     plt.show()
#     print(f"temp_df:\n{temp_df.to_string()}")
#     min_val=df_d.get_group(df_days[0])['total_100_events_time(hr)'].min()
    

In [ ]:
# min_val=df_f['total_100_events_time(hr)'].min()
# min_val

In [ ]:
# min_idx=df_f['total_100_events_time(hr)'].argmin()
# min_idx

In [ ]:
# min_activity=df_f.loc[min_idx]
# min_activity

In [ ]:
#group by
#grouped=df_f.groupby('initial_activity(Bq)')

In [ ]:
# group_list=list(grouped.groups)
# group_list

In [ ]:
for i in group_list:
    print(f"with activity: {i}")
    g=grouped.get_group(i).plot(x='increment_days(d)',y='total_100_events_time(hr)',kind='scatter',figsize=(30,10),grid=True,legend=True,logy=True,title=f'initial_activity:{i} Bq')
    plt.show()

In [ ]:
# for i in group_list:
#     print(grouped.get_group(i).to_string())

In [ ]:
# for i in group_list:
#     print(grouped.get_group(i).to_string())

In [ ]:
for i in [1,5,15,30,60,120]:
    get_all_single(increment_days=i)

# STOP HERE

In [ ]:
# ax1=df_1['decay_rate(Bq)'].plot()
# ax1.set_xlabel('days')
# ax1.set_ylabel('decay_rate [Bq]')
# plt.plot()
# plt.show()

# ax2=df_1['optimal_rate(Hz)'].plot()
# ax2.set_xlabel('days')
# ax2.set_ylabel('Optimal_rate [Hz]')
# plt.plot()
# plt.show()


# ax3=df_1['100_events_time(hr)'].plot()
# ax3.set_xlabel('days')
# ax3.set_ylabel('100_events_time [Hr]')
# plt.plot()
# plt.show()


In [ ]:
#get_realistic_rate(200.0,gamma,alpha,t)

In [ ]:
#lists

# yrs=5         #5 years
# increment_days=1  #increment days



# days=[];rate=[];time=[]

# # 5yrs increment by 1 day
# print(f"""
# ===================================================
# Th228 half life       : {th228_half_life} d
# Th228 optimal activity: {th228_initial_activity} Bq
# ===================================================
# """)

# print(f"""
# Total time: {yrs} yrs
# increment : {increment_days} d

# """)
# for i in range(1,365*yrs,increment_days):
#     #print(f"working for {i} days")
#     rate_realistic=get_rate_after(th228_initial_activity,th228_half_life,i)
#     N_events=1000
#     to_hrs=60*60
#     t_realistic=N_events/(rate_realistic*to_hrs)
#     days.append(i);rate.append(rate_realistic);time.append(t_realistic)
#     if (i-increment_days)/increment_days==0:
#         print(f"""
#         day       : {i} day(s)
#         decay rate: {rate_realistic} Hz
#         time      : {t_realistic} hr
#         """)